# Last.fm — Data Quality Notebook (Scala + Spark)

**Goal:** Manually verify (and document) data quality for the Last.fm 1K users dataset using **Scala + Spark**.

**What this notebook covers:**
1. Ingestion with **explicit schema** and **UTC timezone**.
2. **Robust timestamp parsing** and counting invalid rows.
3. **Key normalization**: prefer `track_id` (MBID), fallback to `artist_name — track_name`.
4. **String sanitization** (trim, remove control chars).
5. **Data quality metrics** (read/valid/dropped rows, % missing MBIDs).
6. **Policy for empty fields** (user_id / artist_name / track_name).
7. **Semantic rule checks** for session gaps: **= 20 min** vs **> 20 min**.
8. **Duplicate detection** (same user, same timestamp, same track).
9. *(Optional)* **Deequ** constraints (nullability, uniqueness).

**Tested/compatible with:** Scala **2.12** and Spark **3.5.x**. Adjust library coordinates accordingly if you add optional Deequ.

In [2]:
import $ivy.`org.apache.spark::spark-sql:3.5.1`
import $ivy.`org.plotly-scala::plotly-almond:0.8.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
init()

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.logging.log4j.{LogManager, Level => LogLevel}
import org.apache.logging.log4j.core.Logger

import org.apache.spark.sql.types._

// Ajusta o nível de log para suprimir INFO antes de iniciar o Spark
System.setProperty("log4j2.level", "WARN")

val spark = SparkSession.builder()
  .appName("LastFM-DataCleaning")
  .master("local[*]")
  .config("spark.sql.shuffle.partitions", "4")
  .getOrCreate()

spark.conf.set("spark.sql.session.timeZone", "UTC")

// Reduz log para ERROR em loggers Spark e Hadoop
Seq(
  "org.apache.spark",
  "org.apache.spark.sql.execution",
  "org.apache.spark.storage",
  "org.apache.hadoop",
  "org.spark_project"
).foreach { name =>
  LogManager.getLogger(name).asInstanceOf[Logger].setLevel(LogLevel.ERROR)
}

LogManager.getRootLogger.asInstanceOf[Logger].setLevel(LogLevel.ERROR)

import spark.implicits._

// Defaults
var INPUT_PATH: String = "/Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-timestamp-artid-artname-traid-traname.tsv" // main play logs TSV
var PROFILE_PATH: String = "/Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-profile.tsv"
val SAMPLE_ROWS = 20 // number of rows to show in samples

println(s"Using INPUT_PATH  = ${INPUT_PATH}")
println(s"Using PROFILE_PATH = ${PROFILE_PATH}")

Using INPUT_PATH  = /Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-timestamp-artid-artname-traid-traname.tsv
Using PROFILE_PATH = /Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-profile.tsv


import $ivy.$
import $ivy.$
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.logging.log4j.{LogManager, Level => LogLevel}
import org.apache.logging.log4j.core.Logger
import org.apache.spark.sql.types._
res2_9: String = "WARN"
spark: SparkSession = org.apache.spark.sql.SparkSession@74578c3
import spark.implicits._
INPUT_PATH: String = "/Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-timestamp-artid-artname-traid-traname.tsv"
PROFILE_PATH: String = "/Users/Felipe/lastfm/data/lastfm/lastfm-dataset-1k/userid-profile.tsv"
SAMPLE_ROWS: Int = 20

## 1) Ingestion with explicit schema
**Purpose:** Avoid incorrect type inference and lock expected column order.

**Columns:**
 - `user_id` (String, not null)
 - `ts_str` (String, not null) — raw timestamp to be parsed later
 - `artist_id` (String, nullable) — MBID
 - `artist_name` (String, nullable)
 - `track_id` (String, nullable) — MBID
 - `track_name` (String, nullable)

In [3]:
val schema = StructType(Seq(
  StructField("user_id", StringType, nullable = false),
  StructField("ts_str", StringType, nullable = false),
  StructField("artist_id", StringType, nullable = true),
  StructField("artist_name", StringType, nullable = true),
  StructField("track_id", StringType, nullable = true),
  StructField("track_name", StringType, nullable = true)
))

val rawDf = spark.read
  .option("sep", "\t")
  .option("header", "false")
  .schema(schema)
  .csv(INPUT_PATH)

val rowsRead = rawDf.count
println(s"Rows read (raw): ${rowsRead}")
rawDf.show(SAMPLE_ROWS, truncate = false)
rawDf.printSchema()

cmd3.sc:16: Auto-application to `()` is deprecated. Supply the empty argument list `()` explicitly to invoke method count,
or remove the empty argument list from its definition (Java-defined methods are exempt).
In Scala 3, an unapplied method like this will be eta-expanded into a function. [quickfixable]
val rowsRead = rawDf.count
                     ^


Rows read (raw): 19150868
+-----------+--------------------+------------------------------------+---------------+------------------------------------+------------------------------------------+
|user_id    |ts_str              |artist_id                           |artist_name    |track_id                            |track_name                                |
+-----------+--------------------+------------------------------------+---------------+------------------------------------+------------------------------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e99-8624-24a6e15f133a|Deep Dish      |NULL                                |Fuck Me Im Famous (Pacha Ibiza)-09-28-2007|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一       |NULL                                |Composition 0919 (Live_2009_4_15)         |
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一       |NULL                                |Mc2 (Live_2009_4_1

schema: StructType = Seq(
  StructField(
    name = "user_id",
    dataType = StringType,
    nullable = false,
    metadata = {}
  ),
  StructField(
    name = "ts_str",
    dataType = StringType,
    nullable = false,
    metadata = {}
  ),
  StructField(
    name = "artist_id",
    dataType = StringType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "artist_name",
    dataType = StringType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "track_id",
    dataType = StringType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "track_name",
    dataType = StringType,
    nullable = true,
    metadata = {}
  )
)
rawDf: DataFrame = [user_id: string, ts_str: string ... 4 more fields]
rowsRead: Long = 19150868L

## 2) Robust timestamp parsing & invalid row accounting
**Purpose:** Parse `ts_str` into a proper `timestamp` column (`ts`) and count invalid rows. Keep only rows with a valid timestamp.

**Format used:** `yyyy-MM-dd'T'HH:mm:ss` (UTC)

In [5]:
val DT_FMT = "yyyy-MM-dd'T'HH:mm:ss"

val withTsDf = rawDf
  .withColumn("ts", to_timestamp(col("ts_str"), DT_FMT))
  .drop("ts_str")

val invalidTsCount = withTsDf.filter(col("ts").isNull).count()
val validTsDf = withTsDf.filter(col("ts").isNotNull)

println(s"Invalid due to timestamp parse: ${invalidTsCount}")
println(s"Valid rows after ts parse: ${validTsDf.count()}")

validTsDf.show(SAMPLE_ROWS, truncate = false)

17:58:45.145 [Executor task launch worker for task 4.0 in stage 5.0 (TID 41)] ERROR org.apache.spark.executor.Executor - Exception in task 4.0 in stage 5.0 (TID 41)
org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2008-09-27T06:42:54Z' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54) ~[spark-sql-api_2.13-3.5.1.jar:3.5.1]
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48) ~[spark-sql-api_2.13-3.5.1.jar:3.5.1]
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218) ~[spark-sql-api_2.13-3.5.1

org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 5.0 failed 1 times, most recent failure: Lost task 4.0 in stage 5.0 (TID 41) (192.168.88.39 executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2008-09-27T06:42:54Z' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$ch

## 3) Key normalization (MBID preferred; fallback to `artist_name — track_name`)
**Purpose:** Build a stable `track_key` used for counts/joins even when MBIDs are missing.

**Rule:** If `track_id` (MBID) is present & non-empty, use it; otherwise use `${artist_name} — ${track_name}` with nulls replaced by `?`.

In [ ]:
val normalizedDf = validTsDf.withColumn(
  "track_key",
  when(col("track_id").isNotNull && length(col("track_id")) > 0, col("track_id"))
    .otherwise(concat_ws(" — ", coalesce(col("artist_name"), lit("?")), coalesce(col("track_name"), lit("?"))))
)

normalizedDf.select("user_id","ts","artist_id","artist_name","track_id","track_name","track_key")
  .show(SAMPLE_ROWS, truncate = false)

## 4) String sanitization
**Purpose:** Remove control characters and trim whitespace from key string fields.

In [ ]:
val sanitizeUdf = udf { s: String =>
  if (s == null) null
  else s.replaceAll("\\p{Cntrl}", "").trim
}

val cleanDf = normalizedDf
  .withColumn("artist_name", sanitizeUdf(col("artist_name")))
  .withColumn("track_name", sanitizeUdf(col("track_name")))
  .withColumn("user_id", sanitizeUdf(col("user_id")))

cleanDf.select("user_id","artist_name","track_name").show(SAMPLE_ROWS, truncate = false)

## 5) Data Quality metrics summary
**Purpose:** Summarize read/valid/dropped counts and percent of missing MBIDs.

In [ ]:
val totalRead = rowsRead
val totalValid = cleanDf.count()
val totalDropped = totalRead - totalValid
val missingTrackId = cleanDf.filter(col("track_id").isNull || length(col("track_id")) === 0).count()
val pctMissingTrackId = if (totalValid == 0) 0.0 else missingTrackId.toDouble / totalValid * 100.0

println(f"rows_read            : ${totalRead}%d")
println(f"rows_valid           : ${totalValid}%d")
println(f"rows_dropped         : ${totalDropped}%d")
println(f"missing_track_id     : ${missingTrackId}%d")
println(f"pct_missing_track_id : ${pctMissingTrackId}%.2f%%")

Seq(
  ("rows_read", totalRead.toString),
  ("rows_valid", totalValid.toString),
  ("rows_dropped", totalDropped.toString),
  ("missing_track_id", missingTrackId.toString),
  ("pct_missing_track_id", f"${pctMissingTrackId}%.2f%%")
).toDF("metric","value").show(truncate = false)

## 6) Empty-field policies
**Purpose:** Inspect how many rows have empty `user_id`, `artist_name`, or `track_name` and decide whether to drop or impute.

**Recommendation:** Drop rows with empty `user_id` and either drop or mark unknown `artist/track` depending on downstream needs (document in README).


In [ ]:
val emptyUser = cleanDf.filter(coalesce(col("user_id"), lit("")) === "").count()
val emptyArtist = cleanDf.filter(coalesce(col("artist_name"), lit("")) === "").count()
val emptyTrack = cleanDf.filter(coalesce(col("track_name"), lit("")) === "").count()

println(s"Rows with empty user_id     : $emptyUser")
println(s"Rows with empty artist_name : $emptyArtist")
println(s"Rows with empty track_name  : $emptyTrack")

val DROP_EMPTY = true // toggle this policy if needed
val dqDf = if (DROP_EMPTY) {
  cleanDf.filter(col("user_id") =!= "" && col("artist_name") =!= "" && col("track_name") =!= "")
} else cleanDf

println(s"Rows after empty-field policy: ${dqDf.count()}")

## 7) Semantic rule — session gap boundary (=20 vs >20 minutes)
**Purpose:** Build a tiny synthetic dataset to verify the session split rule at 20 minutes.

**Rule:**
- gap **≤ 20** minutes → same session
- gap **> 20** minutes → new session

In [ ]:
import java.sql.Timestamp

def ts(s: String) = Timestamp.valueOf(s.replace("T", " "))

val sessionCheck = Seq(
  ("u1", ts("2023-01-01T10:00:00"), "A"),
  ("u1", ts("2023-01-01T10:20:00"), "B"), // exactly 20 min → SAME session
  ("u1", ts("2023-01-01T10:40:01"), "C"), // > 20 min → NEW session
  ("u2", ts("2023-01-01T09:00:00"), "X"),
  ("u2", ts("2023-01-01T09:15:00"), "Y")  // < 20 min → SAME session
).toDF("user_id","ts","track_key")

sessionCheck.orderBy("user_id","ts").show(truncate = false)

import org.apache.spark.sql.expressions.Window
val w = Window.partitionBy("user_id").orderBy(col("ts").asc)

val prevTs = lag(col("ts"), 1).over(w)
val gapSec = (col("ts").cast("long") - prevTs.cast("long"))
val gapMin = when(prevTs.isNull, lit(null).cast("double")).otherwise(gapSec / 60.0)

val withGaps = sessionCheck
  .withColumn("prev_ts", prevTs)
  .withColumn("gap_minutes", gapMin)
  .withColumn("is_new_session", when(prevTs.isNull, 1).when(col("gap_minutes") > 20.0, 1).otherwise(0))
  .withColumn("session_seq", sum(col("is_new_session")).over(w))

withGaps.orderBy("user_id","ts").show(truncate = false)

## 8) Duplicate detection
**Purpose:** Identify potential duplicates defined as **same user, same timestamp, same track**.

**Action:** Count duplicates and preview a few; decide whether to drop or keep (document in README).

In [ ]:
val dupCols = Seq("user_id","ts","track_key")

val dupCounts = dqDf
  .groupBy(dupCols.map(col): _*)
  .agg(count(lit(1)).alias("cnt"))
  .filter(col("cnt") > 1)

val totalDupRows = if (dupCounts.head(1).isEmpty) 0L else dupCounts.select(sum("cnt")).first.getLong(0)

println(s"Distinct duplicate keys: ${dupCounts.count()}")
println(s"Total duplicated rows   : ${totalDupRows}")

dupCounts.orderBy(col("cnt").desc).show(20, truncate = false)

## 9) (Optional) Deequ constraints
**Purpose:** Validate constraints like nullability and uniqueness using **AWS Deequ**. This section is optional and requires adding Deequ as a dependency.

**How to enable:**
- Add library: `"com.amazon.deequ" %% "deequ" % "2.0.7-spark-3.3"` (or a version compatible with your Spark).
- Then run checks such as: not-null on `user_id`, timestamp; uniqueness on `(user_id, ts)` if required.

**Note:** The exact coordinates vary by Spark/Scala versions; confirm compatibility.

## 10) Summary & Next steps
**What we verified:**
- Explicit schema & UTC timezone.
- Timestamp parsing with invalid-row accounting.
- Track key normalization (MBID preferred, fallback safe).
- String sanitization.
- DQ metrics (read/valid/dropped, % missing MBIDs).
- Empty-field policies and their impact.
- Semantic rule at the 20-minute boundary (session split).
- Duplicate detection and preview.
 
**Next steps (suggested):**
- Decide and enforce final policies (drop vs. impute) and document in README.
- Persist cleaned datasets to a curated zone (e.g., Parquet, partitioned).
- Integrate this DQ notebook in CI (smaller synthetic samples) to prevent regressions.
- (Optional) Add Deequ checks into automated pipelines for continuous monitoring.

## 11) Save curated data (example)
**Purpose:** Demonstrate how to persist the cleaned DataFrame into Parquet format with partitioning.

In [ ]:
val OUTPUT_PATH = "/Users/Felipe/lastfm/output/curated"

dqDf.write
  .mode("overwrite")
  .partitionBy("user_id")
  .parquet(OUTPUT_PATH)

println(s"Curated dataset saved to ${OUTPUT_PATH}")

## 12) Export DQ metrics
**Purpose:** Persist the summary metrics into a CSV/Parquet for reporting.

In [ ]:
val dqMetrics = Seq(
  ("rows_read", rowsRead.toString),
  ("rows_valid", totalValid.toString),
  ("rows_dropped", totalDropped.toString),
  ("missing_track_id", missingTrackId.toString),
  ("pct_missing_track_id", f"${pctMissingTrackId}%.2f%%")
).toDF("metric","value")

val METRICS_PATH = "/Users/Felipe/lastfm/output/metrics"

dqMetrics.write.mode("overwrite").option("header","true").csv(METRICS_PATH)

println(s"DQ metrics exported to ${METRICS_PATH}")

## 13) Join with profile data (optional)
**Purpose:** Combine plays with user profile info for enriched analysis.

In [ ]:
val profileSchema = StructType(Seq(
  StructField("user_id", StringType, nullable = false),
  StructField("gender", StringType, nullable = true),
  StructField("age", IntegerType, nullable = true),
  StructField("country", StringType, nullable = true),
  StructField("signup", StringType, nullable = true)
))

val profileDf = spark.read
  .option("sep", "	")
  .option("header", "false")
  .schema(profileSchema)
  .csv(PROFILE_PATH)

val enrichedDf = dqDf.join(profileDf, Seq("user_id"), "left")

enrichedDf.show(SAMPLE_ROWS, truncate = false)

## 14) Wrap-up
**Key takeaways:**
- Data quality checks caught invalid timestamps, missing IDs, and empty fields.
- Clear policies and documented assumptions make the pipeline reproducible.
- Outputs (curated plays + metrics + profiles) are ready for downstream use in sessionization and top-track analysis.

**End of Notebook**